<h1 style="background-color:orange;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;">Introduction</h1>

<h1 style="font-family:Georgia;font-size:175%;text-align:center">Through this data we will try to know more about strokes and Make a model to try to predict strokes</h1>
<h1 style="font-family:Georgia;font-size:150%;text-align:left">first what is a stroke?</h1>

*  Stroke is a medical emergency. A stroke occurs when blood flow to a part of your brain is interrupted or reduced, preventing brain tissue from getting oxygen and nutrients. Brain cells begin to die within minutes 


<h1 style="background-color:orange;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;">Dataset Information</h1>

1) id: unique identifier

2) gender: "Male", "Female" or "Other" <br>

3) age: age of the patient <br>

4) hypertension: 0 if the patient doesn't have hypertension, 1 if the patient has hypertension <br>

5) heart_disease: 0 if the patient doesn't have any heart diseases, 1 if the patient has a heart disease <br>

6) ever_married: "No" or "Yes" <br>

7) work_type: "children", "Govt_jov", "Never_worked", "Private" or "Self-employed" <br>

8) Residence_type: "Rural" or "Urban" <br>

9) avg_glucose_level: average glucose level in blood <br>

10) bmi: body mass index <br>

11) smoking_status: "formerly smoked", "never smoked", "smokes" or "Unknown"* <br>

12) stroke: 1 if the patient had a stroke or 0 if not <br>

*Note: "Unknown" in smoking_status means that the information is unavailable for this patient

<h1 style="background-color:orange;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;">Importing Libraries</h1>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import plotly.express as ex
import plotly.graph_objs as go
import plotly.offline as pyo
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

<h1 style="background-color:orange;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;">Read & Explore</h1>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
Inputpath = ""
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        Inputpath = os.path.join(dirname, filename)
        print(Inputpath)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
stroke = pd.read_csv(Inputpath)

In [ ]:
stroke.shape

In [ ]:
stroke.head()

In [ ]:
stroke.describe()

In [ ]:
stroke.info()

#### * info() says that bmi column have (5110-4909) = 201 null value / missing value

<h1 style="background-color:orange;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;">Data Preprocessing</h1>


## Data Summary ( Check for missing values ) 

In [ ]:
print("Rows            :", stroke.shape[0])
print("Columns         :", stroke.shape[1])
print ("Features       :" , stroke.columns.tolist())
print("Missing Values  :", stroke.isnull().sum().values.sum())
print("Missing Values  :\n", stroke.isnull().sum())

In [ ]:
# A really fantsatic and intelligent way to deal with blanks, from Thoman Konstantin in: https://www.kaggle.com/thomaskonstantin/analyzing-and-modeling-stroke-data
DT_bmi_pipe = Pipeline( steps=[ 
                               ('scale',StandardScaler()),
                               ('lr',DecisionTreeRegressor(random_state=42))
                              ])

#take the copy of df
X = stroke[['age','gender','bmi']].copy()

#change the categorical variable to numaric
X.gender = X.gender.replace({'Male':0,'Female':1,'Other':-1}).astype(np.uint8)

#filterout only missing variables
Missing = X[X.bmi.isna()]

#take non null values
X = X[~X.bmi.isna()]
#Take only "bmi" values
Y = X.pop('bmi')

#performing fit transformation on X and Y, As we take bmi data in Y hence X dont have bmi data it only have age and gender
DT_bmi_pipe.fit(X,Y)

#With the dession Tree regrassor we have predict the bmi
predicted_bmi = pd.Series(DT_bmi_pipe.predict(Missing[['age','gender']]),index=Missing.index)

#assigning the predictive BMI to our df
stroke.loc[Missing.index,'bmi'] = predicted_bmi


#### Initially, the dataset had 201 samples with absent BMI value; rather than imputing it naively with the mean or the median, we used a simple decision tree model which based on the age and gender of all other samples gave us a fair prediction for the missing values.

In [ ]:
print("Missing Values  :", stroke.isnull().sum().values.sum())

<a id="1"></a>

<h1 style="background-color:orange;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;">Exploratory Data Analysis</h1>


# Data Visualization

In [ ]:
data = stroke.stroke.value_counts()
print(data)

#creating Pie Chart
fig = ex.pie(stroke,names='stroke')
fig.update_layout(title='<b>Stroke Samples<b>')
fig.show()



We are clearly dealing with an imbalanced dataset, and later in our pipeline, we will have to deal with this problem potentially with upscaling; that way, we give our models a better chance of learning the small details which define potential stroke individuals.


<a id="1"></a>

<h1 style="background-color:orange;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;">Data Visualization</h1>


#### Finding Categorical and Numarical Variable

In [ ]:
#Categorical Variable
print("Gender         :",stroke.gender.unique())
print("ever_married   :",stroke.ever_married.unique())
print("work_type      :",stroke.work_type.unique())
print("Residence_type :",stroke.Residence_type.unique())
print("smoking_status :",stroke.smoking_status.unique())

In [ ]:
#Numaric Variable
print("hypertension      :",stroke.hypertension.unique())
print("heart_disease     :",stroke.heart_disease.unique())
print("age               :",stroke.age.unique())
print("avg_glucose_level :",stroke.avg_glucose_level.unique())
print("bmi               :",stroke.bmi.unique())

## Numaric Variables

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 10))
stroke.plot(kind="hist", y="age", bins=70, color="b", ax=axes[0][0])
stroke.plot(kind="hist", y="bmi", bins=100, color="r", ax=axes[0][1])
stroke.plot(kind="hist", y="avg_glucose_level", bins=100, color="orange", ax=axes[1][0])
plt.show()

## Observation

* <b>Age</b>: Distribution is normal distribution
* <b>BMI</b>: Looks like it have outliers. (See below box plot)
* <b>Avg glucose level</b>: distribution is semi Normal distribution but as the normal avg of blood in sugar is less than 140, which will not be good for this feature as most of the data are below 150. hence we will not found proper correlation value between diabetes and strokes

## Univariate  Analysis

## BMI Box Plot

In [ ]:
sns.boxplot(data=stroke, y='bmi')
plt.title('Boxplot of bmi')
plt.show()

### Boxplot of Average Glucose Level

In [ ]:
sns.boxplot(data=stroke, y='avg_glucose_level')
plt.title('Boxplot of avg_glucose_level')
plt.show()

### Boxplot of age

In [ ]:
sns.boxplot(data=stroke, y='age')
plt.title('Boxplot of age')
plt.show()

## Gender

In [ ]:
plt.figure(figsize=(6,5))
sns.countplot(data=stroke,x='gender')

In [ ]:
fig, ax =plt.subplots(3,2, figsize=(12, 10))
sns.countplot(data=stroke, x="hypertension", ax=ax[0][0])
sns.countplot(data=stroke, x="ever_married", ax=ax[0][1])
sns.countplot(data=stroke, x="work_type", ax=ax[1][0])
sns.countplot(data=stroke, x="heart_disease", ax=ax[1][1])
sns.countplot(data=stroke, x="Residence_type", ax=ax[2][0])
sns.countplot(data=stroke, x="smoking_status", ax=ax[2][1])
fig.show()

<h1 style="font-family:Georgia;font-size:175%;text-align:left">Stroke or not in Categorical Features</h1>

In [ ]:
#Good One
cat_stroke = stroke[['gender','Residence_type','smoking_status','stroke']]
summary = pd.concat([pd.crosstab(cat_stroke[x], cat_stroke.stroke) for x in cat_stroke.columns[:-1]], keys=cat_stroke.columns[:-1])
summary

## Bivariate Analysis

In [ ]:
#We are taking only stroke data
strokePositive=stroke.loc[stroke['stroke']==1]

In [ ]:
def print_percentage(plots, pltText):
    for bar in plots.patches:
        plots.annotate(f'{round(bar.get_height()/len(strokePositive)*100,2)} %', xy=(bar.get_x() + bar.get_width() / 2,  
                   bar.get_height()), ha='center', va='center', size=13, xytext=(0, 8), textcoords='offset points')
    
    plt.title(pltText)

<h1 style="font-family:Georgia;font-size:175%;text-align:left">Stroke/Ever Married</h1>

In [ ]:
plt.figure(figsize=(10,5))
plots = sns.countplot(x='ever_married',palette='inferno', data=strokePositive);

print_percentage(plots, "Effect of 'Ever Married' on Stroke\n")

#### Observation: 88.35% of Married person exposed to more stroke

<h1 style="font-family:Georgia;font-size:175%;text-align:left">Stroke/Work Type</h1>

In [ ]:
plt.figure(figsize=(10,5))
plots = sns.countplot(data=strokePositive,x='work_type',palette='cool');

print_percentage(plots,"Effect of 'Work Type' on Stroke\n")

#### Observation: 59.85% Private employee exposed to more stroke

<h1 style="font-family:Georgia;font-size:175%;text-align:left">Stroke/Smoking Status</h1>

In [ ]:
plt.figure(figsize=(10,5))
plots= sns.countplot(data=strokePositive,x='smoking_status',palette='autumn');

print_percentage(plots,"Effect of 'Smoking Status' on Stroke\n")


#### Observation: both formal smoker (28.11%) or regular smoker (16.87%) increases risk of the stroke but I am confused with never smoked data (36.14%)

<h1 style="font-family:Georgia;font-size:175%;text-align:left">Residence Type</h1>

In [ ]:
plt.figure(figsize=(10,5))
plots= sns.countplot(data=strokePositive,x='Residence_type',palette='Greens');

print_percentage(plots, "Effect of 'Residence Type' on Stroke\n")

#### Observation: As both the data are almost same hence we can say that 'Residence Type' has no impact on the stroke.

<h1 style="font-family:Georgia;font-size:175%;text-align:left">Stroke/Heart Disease</h1>

In [ ]:
plt.figure(figsize=(10,5))
plots = sns.countplot(data=strokePositive,x='heart_disease',palette='Reds');

print_percentage(plots, "Effect of 'Heart Disease' on Stroke\n")


#### Observation: 81.12% of people who have had a stroke do not have any heart disease.

<h1 style="font-family:Georgia;font-size:175%;text-align:left">Stroke/Hypertension</h1>

In [ ]:
plt.figure(figsize=(10,5))
plots = sns.countplot(data=strokePositive,x='hypertension',palette='Pastel2');

print_percentage(plots,"Effect of Hypertension on Stroke\n")


#### Observation: 73.49% of strok cases They had hypertension

<h1 style="font-family:Georgia;font-size:175%;text-align:left">Stroke Or not Pair Grid</h1>

In [ ]:
stroke.tail()

In [ ]:
sns.set(style="ticks");
pal = ["#FA5858", "#58D3F7"]

sns.pairplot(stroke, hue="stroke", palette=pal);
plt.title("stroke");

<h1 style="font-family:Georgia;font-size:175%;text-align:left">Heatmap Correlation</h1>

In [ ]:
plt.figure(figsize=(15,7))
sns.heatmap(stroke.corr(),annot=True);

### Observation:
We generally consider correlations above 0.75 to be relatively strong; correlations between 0.45 and 0.75 are moderate, and those below 0.45 are considered weak.<br>
In this case, we cannot see correlations above 0.35. For this kind of data, we are considering correlations above 0.25 to be relatively strong; correlations between 0.17 and 0.24 are moderate, and those below 0.17 are considered weak. <br>
<b>As 0.046 is very week hence we can say there is no correlation between stroke and BMI.<b>

<h1 style="background-color:orange;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;">Data preprocessing</h1>

# Removing Outliers and Redundant Columns

In [ ]:
#need to remove 
stroke.head()

In [ ]:
stroke.drop(labels='id', axis=1, inplace=True)

In [ ]:
stroke.head()

In [ ]:
bmi_outliers=stroke.loc[stroke['bmi']>50]
bmi_outliers['bmi'].shape

In [ ]:
# mean with outliers 
print(bmi_outliers['stroke'].value_counts())

In [ ]:
stroke["bmi"] = stroke["bmi"].apply(lambda x: 50 if x>50 else x)
stroke["bmi"] = stroke["bmi"].fillna(28.4)

In [ ]:
stroke.shape

In [ ]:
print ("Missing values :  ", stroke.isnull().sum().values.sum())

<h1 style="font-family:Georgia;font-size:175%;text-align:left">Encoding Categorical Features</h1>

In [ ]:
stroke["Residence_type"] = stroke["Residence_type"].apply(lambda x: 1 if x=="Urban" else 0)
stroke["ever_married"] = stroke["ever_married"].apply(lambda x: 1 if x=="Yes" else 0)
stroke["gender"] = stroke["gender"].apply(lambda x: 1 if x=="Male" else 0)

 
stroke = pd.get_dummies(data=stroke, columns=['smoking_status'])
stroke = pd.get_dummies(data=stroke, columns=['work_type'])


In [ ]:
stroke

<h1 style="font-family:Georgia;font-size:175%;text-align:left">Scaling The variance in Features</h1>

In [ ]:
std=StandardScaler()
columns = ['avg_glucose_level','bmi','age']
scaled = std.fit_transform(stroke[['avg_glucose_level','bmi','age']])
scaled = pd.DataFrame(scaled,columns=columns)
stroke=stroke.drop(columns=columns,axis=1)

In [ ]:
stroke=stroke.merge(scaled, left_index=True, right_index=True, how = "left")
stroke

In [ ]:
stroke.info()

<h1 style="background-color:orange;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;">Classification Models</h1>

<h1 style="font-family:Georgia;font-size:175%;text-align:left">Target & Features </h1>

In [ ]:
X = stroke.drop(['stroke'], axis=1).values 
y = stroke['stroke'].values

<h1 style="font-family:Georgia;font-size:175%;text-align:left">Splitting </h1>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

<h1 style="font-family:Georgia;font-size:175%;text-align:left">SVM </h1>

In [ ]:
svc = SVC(random_state=0)
svc.fit(X_train, y_train)
#== 
#Score 
#== 
svc_score = svc.score(X_train, y_train)
svc_test = svc.score(X_test, y_test)
#== 
#testing model 
#== 
y_pred = svc.predict(X_test)
#== 
#evaluation
#== 
print('Training Score:',svc_score)
print('Testing Score:',svc_test)
svc_accuracy_score = accuracy_score(y_test,y_pred)
print('Accuracy Score:',svc_accuracy_score)
#=== 
#Confusion Matrix 
plt.figure(figsize=(8,5))

cm = confusion_matrix(y_test,y_pred)
conf_matrix = pd.DataFrame(data=cm,columns = ['Predicted Negative', 'Predicted Positive'],
                           index = ['Actual Negative', 'Actual Positive'])

sns.heatmap(conf_matrix, annot=True,fmt='d',cmap="BuPu");

### Sensitivity & Specificity

![](https://2.bp.blogspot.com/-EvSXDotTOwc/XMfeOGZ-CVI/AAAAAAAAEiE/oePFfvhfOQM11dgRn9FkPxlegCXbgOF4QCLcBGAs/s1600/confusionMatrxiUpdated.jpg)

In [ ]:
TN=cm[0,0]
TP=cm[1,1]
FN=cm[1,0]
FP=cm[0,1]
svm_sensitivity=TP/float(TP+FN)
svm_specificity=TN/float(TN+FP)

print(f'TP: {TP}, FN: {FN}, FP: {FP}, TN: {TN},')

print('\n\nThe acuuracy of the model = TP+TN/(TP+TN+FP+FN) =       ',(TP+TN)/float(TP+TN+FP+FN),'\n',

'The Missclassification = 1-Accuracy =                  ',1-((TP+TN)/float(TP+TN+FP+FN)),'\n',

'Sensitivity or True Positive Rate = TP/(TP+FN) =       ',svm_sensitivity,'\n',

'Specificity or True Negative Rate = TN/(TN+FP) =       ',svm_specificity,'\n')

<h1 style="font-family:Georgia;font-size:175%;text-align:left">Random Forest Classifier</h1>

In [ ]:
forest = RandomForestClassifier(n_estimators = 100)
#== 
forest.fit(X_train, y_train)
#== 
#Score 
#== 
forest_score = forest.score(X_train, y_train)
forest_test = forest.score(X_test, y_test)
#== 
#testing model 
#== 
y_pred = forest.predict(X_test)
#== 
#evaluation
#== 

print('Training Score:',forest_score)
print('Testing Score:',forest_test)
forest_accuracy_score = accuracy_score(y_test,y_pred)
print('Accuracy Score:',forest_accuracy_score)
#print(cm)

#=== 
#Confusion Matrix 
plt.figure(figsize=(8,5))

cm = confusion_matrix(y_test,y_pred)
conf_matrix = pd.DataFrame(data=cm,columns = ['Predicted Negative', 'Predicted Positive'],
                           index = ['Actual Negative', 'Actual Positive'])
sns.heatmap(conf_matrix, annot=True,fmt='d',cmap="Blues");




### Sensitivity & Specificity

In [ ]:
TN=cm[0,0]
TP=cm[1,1]
FN=cm[1,0]
FP=cm[0,1]
forest_sensitivity=TP/float(TP+FN)
forest_specificity=TN/float(TN+FP)

print('The acuuracy of the model = TP+TN/(TP+TN+FP+FN) =       ',(TP+TN)/float(TP+TN+FP+FN),'\n',

'The Missclassification = 1-Accuracy =                  ',1-((TP+TN)/float(TP+TN+FP+FN)),'\n',

'Sensitivity or True Positive Rate = TP/(TP+FN) =       ',forest_sensitivity,'\n',

'Specificity or True Negative Rate = TN/(TN+FP) =       ',forest_specificity,'\n')

<h1 style="font-family:Georgia;font-size:175%;text-align:left">Logistic Regression</h1>

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

#== 
#Score 
#== 
logistic_score = model.score(X_train, y_train)
logistic_test = model.score(X_test, y_test)

#== 
#testing model 
#== 
y_pred= model.predict(X_test)


#=== 
#Confusion Matrix 
plt.figure(figsize=(8,5))

cm = confusion_matrix(y_test,y_pred)
conf_matrix = pd.DataFrame(data=cm,columns = ['Predicted Negative', 'Predicted Positive'],
                           index = ['Actual Negative', 'Actual Positive'])
sns.heatmap(conf_matrix, annot=True,fmt='d',cmap="Blues");

#== 
#evaluation
#== 
print('Training Score:',logistic_score)
print('Testing Score:',logistic_test)
forest_accuracy_score = accuracy_score(y_test,y_pred)
print('Accuracy Score:',forest_accuracy_score)
print('Classification Report:\n',classification_report(y_test, y_pred))
#print(accuracy_score(y_test,y_pred))


### Sensitivity & Specificity

In [ ]:
TN=cm[0,0]
TP=cm[1,1]
FN=cm[1,0]
FP=cm[0,1]
logistic_sensitivity=TP/float(TP+FN)
logistic_specificity=TN/float(TN+FP)

print('The acuuracy of the model = TP+TN/(TP+TN+FP+FN) =       ',(TP+TN)/float(TP+TN+FP+FN),'\n',

'The Missclassification = 1-Accuracy =                  ',1-((TP+TN)/float(TP+TN+FP+FN)),'\n',

'Sensitivity or True Positive Rate = TP/(TP+FN) =       ',logistic_sensitivity,'\n',

'Specificity or True Negative Rate = TN/(TN+FP) =       ',logistic_specificity,'\n')

<h1 style="font-family:Georgia;font-size:175%;text-align:left">Feature Importance using Logistic Regression</h1>

In [ ]:
features = list(stroke.columns)
#print(features.index('stroke'))
#== 
#Delete target label 
#== 
del features[features.index('stroke')]
features

In [ ]:
importance = model.coef_
# summarize feature importance
for i,v in enumerate(importance[0]):
    print(f'Feature: {i} ==> "{features[i]}", Score: {abs(v)}')

plt.figure(figsize=(20,7))    
sns.barplot(x=[x for x in range(importance.shape[1])], y=importance[0]).set_xticklabels(features, rotation=90)
plt.show()

In [ ]:
dicts = {}
dictsort = {}
lstkey = []
lstvalue = []
    
for i,v in enumerate(importance[0]):
    lstkey.append(features[i])
    lstvalue.append(round(v,2))

# COnvert to dictionary
for key in lstkey:
    for value in lstvalue:
        dicts[key] = abs(value)
        lstvalue.remove(value)
        break

print("Dictionary from lists :\n ",dicts)

dictsort = {k: v for k, v in sorted(dicts.items(), key=lambda item: item[1])}

print("\n\nDictionary in sorted order :\n ",dictsort)


# Summary of Performance

In [ ]:
from prettytable import PrettyTable
table = PrettyTable()
x = PrettyTable()

x.field_names = ['Model', 'Training Score', 'Testing Score', 'Accuracy Score', 'Sensitivity/True Positive Rate', 'Specificity/True Negative Rate']

x.add_row(["SVM", round(svc_score,3), round(svc_test,3), round(svc_accuracy_score,3), round(svm_sensitivity,3), round(svm_specificity,3)])
x.add_row(["Random Forest", round(forest_score,3), round(forest_test,3), round(forest_accuracy_score,3), round(forest_sensitivity,3), round(forest_specificity,3)])
x.add_row(["Logistic Regression", round(logistic_score,3), round(logistic_test,3), round(forest_accuracy_score,3), round(logistic_sensitivity,3), round(logistic_specificity,3)])

print(x)

## Observation
- <b>0%</b> People <u>with</u> disease(Stroke) were correctly identify and <b>100%</b> people <u>without</u> disease(Stroke) were corretly idntify by the <b>SVM</b><br>
- <b>0%</b> People <u>with</u> disease(Stroke) were correctly identify and <b>99.7%</b> people <u>without</u> disease(Stroke) were corretly idntify by the <b>Random Forest</b><br>
- <b>0.01%</b> People <u>with</u> disease(Stroke) were correctly identify and <b>100%</b> people <u>without</u> disease(Stroke) were corretly idntify by the <b>Logistic Regression</b><br><br>
#### From the above statistics it is clear that the models are <b><u>more highly specific than sensitive</u></b>. Models are able to accurately predict the negative cases over the positives.